In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import re
import pandas as pd
import collections
import time
import random

In [10]:
PATH = "C:\Program Files (x86)\chromedriver_win32\chromedriver.exe"
s=Service(PATH)
driver = webdriver.Chrome(service=s)
driver.get('https://www.fotmob.com/')
games = driver.find_elements(By.CLASS_NAME, 'ew7iiy60')
links = [game.get_attribute('href') for game in games]

In [163]:
url = 'https://www.fotmob.com/match/3657231/matchfacts/sassuolo-vs-hellas-verona?date=20220116'
driver = webdriver.Chrome(service=s)
driver.get(url)
mainstats = driver.find_elements(By.XPATH, '(//ul[@class="css-45dl0h-StatGroupContainer e683amr3"])')[0].text
time.sleep(random.randint(0, 4))
possession = driver.find_elements(By.XPATH, '(//div[@class="css-iwgu3j-PossessionWheel e683amr7"])')[0].text
allstats = driver.find_elements(By.XPATH , '(//button[@class="css-1isl7sd-ExpandButton e7i6a0l1"])[2]')[0]
time.sleep(random.randint(0, 4))
#allstats.click()
time.sleep(random.randint(0, 4))
stats = driver.find_elements(By.CLASS_NAME, 'e360fsv1')[1].text
game = driver.find_elements(By.CLASS_NAME, 'css-jkaqxa')[1].text
times = driver.find_elements(By.CLASS_NAME, 'css-xj92le')[1].text
driver.quit()

In [185]:
ms = re.split(pattern = "\\r?\\n", string = mainst)
os = ms[6:12]
os.extend(ms[-9:])

In [186]:
os

['11',
 'Chances created',
 '14',
 '2',
 'Big chances',
 '3',
 '6',
 'Fouls conceded',
 '18',
 '9',
 'Corners',
 '7',
 '0',
 'Offsides',
 '8']

In [92]:
possession, stats, game, times = ebgs(url)

In [77]:
los = re.split(pattern = "\\r?\\n", string = stats)
try:
    los.remove('What is xG?')
    los.remove('Keeper')
    los.remove('Discipline')
except:
    pass


In [82]:

columns = los[1::3]
columns.insert(0, 'Goals')
columns.insert(2, 'Possession')
team1 = los[::3]
team1.insert(0, goal1)
team1.insert(2, pos1)
team2 = los[2::3]
team2.insert(0, goal2)
team2.insert(2, pos2)

In [76]:
pos = possession.split()[-2:]
pos1, pos2 = pos[0], pos[1]
score = game.split()[::2]
goal1, goal2 = score[0], score[1]

In [84]:
cols = ['1 ' + word for word in columns] + ['2 ' + word for word in columns]
gs = team1 + team2
row = {k:v for k, v in zip(cols, gs)}

In [85]:
fd = collections.defaultdict(list)
for d in [row]:
    for key, value in d.items():
        fd[key].append(value)

In [7]:
def ebgs(url):
    driver = webdriver.Chrome(service=s)
    driver.get(url)
    time.sleep(random.randint(0, 4))
    mainstats = driver.find_elements(By.XPATH, '(//ul[@class="css-45dl0h-StatGroupContainer e683amr3"])')[0].text
    possession = driver.find_elements(By.XPATH, '(//div[@class="css-iwgu3j-PossessionWheel e683amr7"])')[0].text
    allstats = driver.find_elements(By.XPATH , '(//button[@class="css-1isl7sd-ExpandButton e7i6a0l1"])[2]')[0]
    allstats.click()
    time.sleep(random.randint(0, 6))
    stats = driver.find_elements(By.CLASS_NAME, 'e360fsv1')[1].text
    game = driver.find_elements(By.CLASS_NAME, 'css-jkaqxa')[1].text
    times = driver.find_elements(By.CLASS_NAME, 'css-xj92le')[1].text
    driver.quit()
    return mainstats, possession, stats, game, times

In [8]:
def getstats(url):
    mainstats, possession, stats, game, times = ebgs(url)
    pos = possession.split()[-2:]
    pos1, pos2 = pos[0], pos[1]
    score = game.split()[::2]
    goal1, goal2 = score[0], score[1]
    score = game.split()[::2]
    goal1 = score[0]
    goal2 = score[1]
    ms = re.split(pattern = "\\r?\\n", string = mainst)
    os = ms[6:12]
    os.extend(ms[-9:])
    los = re.split(pattern = "\\r?\\n", string = stats)
    los.remove('Keeper')
    los.remove('Discipline')
    los.extend(os)
    try:
        los.remove('What is xG?')
    except:
        pass
    columns = los[1::3]
    columns.insert(0, 'Goals')
    columns.insert(2, 'Possession')
    team1 = los[::3]
    team1.insert(0, goal1)
    team1.insert(2, pos1)
    team2 = los[2::3]
    team2.insert(0, goal2)
    team2.insert(2, pos2)
    cols = ['1 ' + word for word in columns] + ['2 ' + word for word in columns]
    gs = team1 + team2
    row = {k:v for k, v in zip(cols, gs)}
    name = url.split('/')[-1]
    row['name'] = name
    return row

In [9]:
x = getstats("https://www.fotmob.com/match/3657225/matchfacts/fiorentina-vs-genoa")

NameError: name 's' is not defined

In [3]:
"https://www.fotmob.com/match/3657225/matchfacts/fiorentina-vs-genoa".split('/')[-1]

'fiorentina-vs-genoa'